In [ ]:
import numpy as np
import pandas as pd
from skimage.feature import blob_log
from laptrack import LapTrack
import napari

# Load images and show in napari

In [ ]:
viewer = napari.Viewer()
images = np.load("data/brownian_particles_images_with_noise.npz")["images"]
viewer.add_image(images)

# Detect spots by Laplacian of Gaussian filter

In [ ]:
spots = []
for frame, image in enumerate(images):
    _spots = blob_log(image, min_sigma=1, max_sigma=3, num_sigma=3,overlap=0.5)
    df = pd.DataFrame(_spots, columns=["y","x","sigma"])
    df["frame"] = frame
    spots.append(df)
spots_df = pd.concat(spots)
viewer.add_points(spots_df[["frame","y","x"]], size=3, edge_color="yellow", face_color="#ffffff00")

# Track by LapTrack

In [ ]:
lt=LapTrack(
    track_cost_cutoff=5**2
)
track_df, _, _ = lt.predict_dataframe(spots_df,["y","x"])
track_df=track_df.reset_index()
viewer.add_tracks(track_df[["track_id","frame","y","x"]])